## Requisitos

    pip install spacy
    python -m spacy download pt_core_news_sm

In [1]:
import re
import spacy
from string import punctuation
from collections import Counter, OrderedDict
from spacy.symbols import ORTH, LEMMA, POS, TAG


nlp = spacy.load('pt_core_news_sm')

for n in nlp('Hoje é terça-feira. Segunda-feira fui ao médico e o Dr. Arnaldo me receitou um remédio.'):
    print(n.text)

Hoje
é
terça
-
feira
.
Segunda
-
feira
fui
a
o
médico
e
o
Dr.
Arnaldo
me
receitou
um
remédio
.




#### Tokenizando com regex pura

In [2]:
import re

PATTERN = re.compile(r'''(?:Dr\.|Dra\.|Sr\.|Sra\.)|[\.,;:]+|\w+\-\w+|\w+''', re.IGNORECASE)

re.findall(PATTERN, 'Hoje é terça-feira. Segunda-feira fui ao médico, e o dr. Arnaldo me receitou um remédio.')

['Hoje',
 'é',
 'terça-feira',
 '.',
 'Segunda-feira',
 'fui',
 'ao',
 'médico',
 ',',
 'e',
 'o',
 'dr.',
 'Arnaldo',
 'me',
 'receitou',
 'um',
 'remédio',
 '.']

<br>
#### Criando casos especiais de tokens no Spacy

In [3]:
special_cases = {
    'segunda-feira': [{ORTH: u'segunda-feira', LEMMA: u'segunda', POS: u'NOUN'}],
    'terça-feira': [{ORTH: u'terça-feira', LEMMA: u'terça', POS: u'NOUN'}],
    'quarta-feira': [{ORTH: u'quarta-feira', LEMMA: u'quarta', POS: u'NOUN'}],
    'quinta-feira': [{ORTH: u'quinta-feira', LEMMA: u'quinta', POS: u'NOUN'}],
    'sexta-feira': [{ORTH: u'sexta-feira', LEMMA: u'sexta', POS: u'NOUN'}],
    'Segunda-feira': [{ORTH: u'Segunda-feira', LEMMA: u'Segunda', POS: u'NOUN'}],
    'Terça-feira': [{ORTH: u'Terça-feira', LEMMA: u'Terça', POS: u'NOUN'}],
    'Quarta-feira': [{ORTH: u'Quarta-feira', LEMMA: u'Quarta', POS: u'NOUN'}],
    'Quinta-feira': [{ORTH: u'Quinta-feira', LEMMA: u'Quinta', POS: u'NOUN'}],
    'Sexta-feira': [{ORTH: u'Sexta-feira', LEMMA: u'Sexta', POS: u'NOUN'}]
}

In [4]:
def add_cases(dicio_cases, nlp_obj):
    
    for key, value in dicio_cases.items():
        nlp_obj.tokenizer.add_special_case(key, value)
        
add_cases(special_cases, nlp)

In [5]:
for n in nlp('Segunda-feira fui ao médico e o Dr. Arnaldo me receitou um remédio.'):
    print(n.text)

Segunda-feira
fui
a
o
médico
e
o
Dr.
Arnaldo
me
receitou
um
remédio
.


In [6]:
from collections import namedtuple


Ner = namedtuple('NER', 'texto label start_char end_char')


class TextNLP:
    
    def __init__(self, text):
        
        assert isinstance(text, str), "'A classe deve ser inicializada com uma string sendo passada por parâmetro'"
        
        if text == '':
            raise ValueError('A string passada não pode estar vazia')
        
        self.text = text
        self.doc = nlp(self.text)
        
        
    def __iter__(self):
        
        return (t.text for t in self.doc)
    
    
    def __repr__(self):
        tokens = [(tok.text, tok.pos_) for tok in self.doc]
        
        return "Texto: %s\n\nTokens: %s\n\nNum Tokens: %d" % (self.text, tokens, len(tokens))
    
    
    @property
    def words(self):
            
        return list(self.__iter__())
    
    
    def words_preprocessing(self, stop_words_remove=True, punctuation_remove=True, lemma=False):
        
        tokens = self.doc
        
        if stop_words_remove and punctuation_remove:
            tokens = (tok for tok in tokens if (not tok.is_punct and not tok.is_stop))
        
        elif punctuation_remove:
            tokens = (tok for tok in tokens if not tok.is_punct)
            
        elif stop_words_remove:
            tokens = (tok for tok in tokens if not tok.is_stop)
            
        else:
            tokens = (tok for tok in tokens)
            
        if lemma:
            tokens = (tok.lemma_ for tok in tokens)
            
        return tokens
    
    
    @property
    def sents(self):
                
        return (sent for sent in self.doc.sents)
    
    
    def ents(self):
        for ent in self.doc.ents:
            yield Ner(ent.text, ent.label_, ent.start_char, ent.end_char)
    
    
    def tokens_counter(self, ordered=True, reverse=True):
        tok = list(self.__iter__())
        
        tok = OrderedDict(Counter(tok))
        
        if ordered:
            tok = OrderedDict(sorted(tok.items(), key=lambda x: x[1], reverse=reverse))
            
        return tok
        

In [7]:
nlp_ = TextNLP('Segunda-feira fui ao médico e o Dr. Arnaldo Costa da Silva da JusBrasil me receitou um remédio. Na próxima semana, irei voltar a Curitiba.')

<br>
### Representação do objeto

In [8]:
nlp_

Texto: Segunda-feira fui ao médico e o Dr. Arnaldo Costa da Silva da JusBrasil me receitou um remédio. Na próxima semana, irei voltar a Curitiba.

Tokens: [('Segunda-feira', 'NOUN'), ('fui', 'VERB'), ('a', 'ADP'), ('o', 'DET'), ('médico', 'NOUN'), ('e', 'CCONJ'), ('o', 'DET'), ('Dr.', 'PROPN'), ('Arnaldo', 'PROPN'), ('Costa', 'PROPN'), ('da', 'PROPN'), ('Silva', 'PROPN'), ('da', 'ADP'), ('JusBrasil', 'PROPN'), ('me', 'PRON'), ('receitou', 'VERB'), ('um', 'DET'), ('remédio', 'NOUN'), ('.', 'PUNCT'), ('Na', 'DET'), ('próxima', 'ADJ'), ('semana', 'NOUN'), (',', 'PUNCT'), ('irei', 'VERB'), ('voltar', 'VERB'), ('a', 'DET'), ('Curitiba', 'PROPN'), ('.', 'PUNCT')]

Num Tokens: 28

### Tokenizando

In [9]:
nlp_.words

['Segunda-feira',
 'fui',
 'a',
 'o',
 'médico',
 'e',
 'o',
 'Dr.',
 'Arnaldo',
 'Costa',
 'da',
 'Silva',
 'da',
 'JusBrasil',
 'me',
 'receitou',
 'um',
 'remédio',
 '.',
 'Na',
 'próxima',
 'semana',
 ',',
 'irei',
 'voltar',
 'a',
 'Curitiba',
 '.']

## NER

In [10]:
for ent in nlp_.ents():
    print(ent, end='\n\n')

NER(texto='Dr. Arnaldo Costa da Silva da JusBrasil', label='PER', start_char=32, end_char=71)

NER(texto='Curitiba', label='LOC', start_char=129, end_char=137)



In [11]:
str(nlp_.doc)[32:58]

'Dr. Arnaldo Costa da Silva'

In [12]:
from spacy import displacy

displacy.render(nlp_.doc, style='ent', jupyter=True)

In [13]:
displacy.render(nlp_.doc, style='dep', jupyter=True, options={'distance': 120})

In [14]:
print('nsubj: ', spacy.explain('nsubj'))
print('MISC: ', spacy.explain('MISC'))
print('LOC: ', spacy.explain('LOC'))
print('PROPN: ', spacy.explain('PROPN'))
print('PRON: ', spacy.explain('PRON'))
print('DET: ', spacy.explain('DET'))

nsubj:  nominal subject
MISC:  Miscellaneous entities, e.g. events, nationalities, products or works of art
LOC:  Non-GPE locations, mountain ranges, bodies of water
PROPN:  proper noun
PRON:  pronoun
DET:  determiner


<br>
### Pre processamento

In [15]:
list(nlp_.words_preprocessing(stop_words_remove=False))

[Segunda-feira,
 fui,
 a,
 o,
 médico,
 e,
 o,
 Dr.,
 Arnaldo,
 Costa,
 da,
 Silva,
 da,
 JusBrasil,
 me,
 receitou,
 um,
 remédio,
 Na,
 próxima,
 semana,
 irei,
 voltar,
 a,
 Curitiba]

In [16]:
# Bom lemmatizer

list(nlp_.words_preprocessing(stop_words_remove=False, lemma=True))

['Segunda',
 'ser',
 'o',
 'o',
 'médico',
 'e',
 'o',
 'Dr.',
 'Arnaldo',
 'Costa',
 'da',
 'Silva',
 'da',
 'JusBrasil',
 'me',
 'receitar',
 'um',
 'remédio',
 'Na',
 'próximo',
 'semana',
 'irar',
 'voltar',
 'o',
 'Curitiba']